In [4]:
import os
import re
import uuid
from typing import List, Tuple

# === 使用者設定 ===
folder = r"D:\ArcGIS\Morpho_Spatial_Demography\split_images\k2b_holes_50cm"  # 資料夾路徑
oids: List[int] = [87, 118, 140, 160, 174, 247, 51, 112, 202, 213, 219, 246, 25, 39, 46, 93, 96, 221]
dry_run = False  # 先預覽；確認無誤後改為 False 進行實際改名

# === 取得資料夾名稱（新檔名前綴） ===
folder_name = os.path.basename(folder.rstrip(r"\/"))

# === 抓取 PNG 檔並解析 mm 後序號 ===
png_files = [f for f in os.listdir(folder) if f.lower().endswith(".png")]

pattern = re.compile(r"mm(\d+)\.png$", re.IGNORECASE)
parsed: List[Tuple[int, str, str]] = []  # (order_num, filename, ext)

for f in png_files:
    m = pattern.search(f)
    if not m:
        # 找不到 mm 序號的檔案跳過（或你也可選擇 raise）
        print(f"⚠️ 檔名無法解析序號（略過）：{f}")
        continue
    order_num = int(m.group(1))
    _, ext = os.path.splitext(f)  # 保留原始副檔名大小寫（.PNG 或 .png）
    parsed.append((order_num, f, ext))

# === 檢查數量是否與 OID 相符 ===
if len(parsed) != len(oids):
    raise ValueError(
        f"檔案可解析數量({len(parsed)}) 與 OID 數量({len(oids)}) 不符，請檢查檔案或 OID 列表。"
    )

# === 依序號排序檔案；OID 也升冪排序 ===
parsed.sort(key=lambda x: x[0])         # 依 mm 後的數字 0,1,2... 排序檔案
sorted_oids = sorted(oids)               # 依 OID 數值升冪排序

# === 預覽對應 ===
plan = []
for (order_num, filename, ext), oid in zip(parsed, sorted_oids):
    new_name = f"{folder_name}_{oid}{ext.lower()}"  # 統一輸出 .png；若要保留原大小寫用 ext
    plan.append((filename, new_name, order_num, oid))

print("📋 對應計畫（原檔 → 新檔 | 原序號 → OID）：")
for old, new, ordnum, oid in plan:
    print(f"{old}  →  {new}   |   mm序號 {ordnum} → OID {oid}")

if dry_run:
    print("\n🧪 目前為預覽模式（dry_run=True），尚未進行改名。確認無誤後將 dry_run 設為 False 再執行。")
else:
    # === 正式改名：先全部暫時改名避免互相覆蓋 ===
    temp_map = {}
    for old, new, _, _ in plan:
        old_path = os.path.join(folder, old)
        temp_name = f"__tmp__{uuid.uuid4().hex}__{old}"
        temp_path = os.path.join(folder, temp_name)
        os.rename(old_path, temp_path)
        temp_map[temp_name] = new

    # === 再從暫時檔名改為最終新檔名 ===
    for temp_name, final_name in temp_map.items():
        temp_path = os.path.join(folder, temp_name)
        final_path = os.path.join(folder, final_name)
        if os.path.exists(final_path):
            # 極端情況：目標檔名已存在，避免覆蓋
            raise FileExistsError(f"目標檔名已存在，已中止：{final_path}")
        os.rename(temp_path, final_path)
        print(f"✅ {temp_name}  →  {final_name}")

    print("\n🎉 全部改名完成！")

📋 對應計畫（原檔 → 新檔 | 原序號 → OID）：
HW_k2b_2508_32604_05mm0.PNG  →  k2b_holes_50cm_25.png   |   mm序號 0 → OID 25
HW_k2b_2508_32604_05mm1.PNG  →  k2b_holes_50cm_39.png   |   mm序號 1 → OID 39
HW_k2b_2508_32604_05mm2.PNG  →  k2b_holes_50cm_46.png   |   mm序號 2 → OID 46
HW_k2b_2508_32604_05mm3.PNG  →  k2b_holes_50cm_51.png   |   mm序號 3 → OID 51
HW_k2b_2508_32604_05mm4.PNG  →  k2b_holes_50cm_87.png   |   mm序號 4 → OID 87
HW_k2b_2508_32604_05mm5.PNG  →  k2b_holes_50cm_93.png   |   mm序號 5 → OID 93
HW_k2b_2508_32604_05mm6.PNG  →  k2b_holes_50cm_96.png   |   mm序號 6 → OID 96
HW_k2b_2508_32604_05mm7.PNG  →  k2b_holes_50cm_112.png   |   mm序號 7 → OID 112
HW_k2b_2508_32604_05mm8.PNG  →  k2b_holes_50cm_118.png   |   mm序號 8 → OID 118
HW_k2b_2508_32604_05mm9.PNG  →  k2b_holes_50cm_140.png   |   mm序號 9 → OID 140
HW_k2b_2508_32604_05mm10.PNG  →  k2b_holes_50cm_160.png   |   mm序號 10 → OID 160
HW_k2b_2508_32604_05mm11.PNG  →  k2b_holes_50cm_174.png   |   mm序號 11 → OID 174
HW_k2b_2508_32604_05mm12.PNG  →  k2b_holes_50

In [26]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
import os
from PIL import Image
import re

# === 使用者設定 ===
image_folder = r"D:\ArcGIS\Morpho_Spatial_Demography\split_images\k2b_holes_50cm"   # PNG 圖片所在資料夾
output_pdf = r"CD:\ArcGIS\Morpho_Spatial_Demography\split_images\k2b_holes_50cm"          # 輸出 PDF 路徑
images = sorted([f for f in os.listdir(image_folder) if f.lower().endswith('.png')])

# === 建立 PDF ===
c = canvas.Canvas(output_pdf, pagesize=letter)
page_width, page_height = letter

# 圖片擺放設定
margin = 0.5 * inch          # 四邊留白
spacing = 0.25 * inch        # 圖之間的距離
usable_height = page_height - 2 * margin
half_height = (usable_height - spacing) / 2    # 每張圖可用高度

# 字體設定
c.setFont("Helvetica", 12)

# 每頁放兩張圖
for i in range(0, len(images), 2):
    for j in range(2):
        idx = i + j
        if idx >= len(images):
            break

        img_name = images[idx]
        img_path = os.path.join(image_folder, img_name)
        img = Image.open(img_path)

        # 保持比例縮放
        img_width, img_height = img.size
        scale = min((page_width - 2 * margin) / img_width, half_height / img_height)
        draw_width = img_width * scale
        draw_height = img_height * scale

        # 第一張在上方、第二張在下方
        if j == 0:
            y_pos = page_height - margin - draw_height
        else:
            y_pos = margin

        x_pos = (page_width - draw_width) / 2
        c.drawImage(img_path, x_pos, y_pos, draw_width, draw_height)

        # 從檔名取出底線後的數字
        match = re.search(r"_([0-9]+)\.png$", img_name, re.IGNORECASE)
        if match:
            label = match.group(1)
            text_x = page_width / 2
            text_y = y_pos - 0.15 * inch   # 圖片下方一點
            c.drawCentredString(text_x, text_y, f"ID: {label}")

    c.showPage()  # 換頁

c.save()
print(f"✅ PDF 已輸出：{output_pdf}")


<class 'ModuleNotFoundError'>: No module named 'reportlab'

In [28]:
!pip install reportlab

Error processing line 1 of C:\Users\keelu\AppData\Local\ESRI\conda\envs\geo\Lib\site-packages\ArcGISPro.pth:
﻿
Fatal Python error: init_import_site: Failed to import the site module
Python runtime state: initialized
Traceback (most recent call last):
  File "C:\Users\keelu\AppData\Local\ESRI\conda\envs\geo\Lib\site.py", line 195, in addpackage
    exec(line)
  File "<string>", line 1, in <module>
  File "C:\Users\keelu\AppData\Local\ESRI\conda\envs\geo\Lib\site-packages\arcpy_init.py", line 6, in <module>
    import re
  File "C:\Users\keelu\AppData\Local\ESRI\conda\envs\geo\Lib\re\__init__.py", line 125, in <module>
    from . import _compiler, _parser
  File "C:\Users\keelu\AppData\Local\ESRI\conda\envs\geo\Lib\re\_compiler.py", line 18, in <module>
    assert _sre.MAGIC == MAGIC, "SRE module mismatch"
AssertionError: SRE module mismatch
﻿
During handling of the above exception, another exception occurred:
﻿
Traceback (most recent call last):
  File "C:\Users\keelu\AppData\Local\ESRI